In [172]:
import cv2
import numpy as np
import sys

sys.path.append('../')
from animatools import animaimg, animavid, animahelpers

In [173]:
def crop_frame_square(image, mask):
    mask = cv2.threshold(mask, 40, 255, cv2.THRESH_BINARY)[1]
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if not contours:
        return image, (0, 0, image.shape[1], image.shape[0])
    x, y, w, h = cv2.boundingRect(contours[0])
    side_length = max(w, h)
    x_center = x + w // 2
    y_center = y + h // 2
    x_square = max(0, x_center - side_length // 2)
    y_square = max(0, y_center - side_length // 2)
    x_square = min(x_square, image.shape[1] - side_length)
    y_square = min(y_square, image.shape[0] - side_length)
    cropped_image = image[y_square:y_square + side_length, x_square:x_square + side_length]
    # print(x_square, y_square, side_length)
    return cropped_image, (x_square, y_square, side_length)

In [174]:
image = cv2.imread('test_images/test/image.jpg')
mask  = cv2.imread('test_images/test/mask.jpg', cv2.IMREAD_GRAYSCALE)
cropped_image, bbox = crop_frame_square(image, mask)
# cv2.imshow('image', cropped_image)
# cv2.waitKey(0)

In [175]:
def restitch_frame(bg_image, fg_image, bbox):
    stitched_image = bg_image.copy()
    # stitched_image = np.zeros_like(bg_image)
    # print(bbox)
    x, y, side_length = bbox
    scaled_fg_image = cv2.resize(fg_image, (side_length, side_length))
    stitched_image[y:y+side_length, x:x+side_length] = scaled_fg_image
    return stitched_image

In [176]:
restitched_image = restitch_frame(image, cropped_image, bbox)
# cv2.imshow('image', restitched_image)
# cv2.waitKey(0)

In [177]:
bbox_array = []

def crop_video_to_mask(input_video_path, mask_video_path, output_video_path, output_bbox_path):
    cap = cv2.VideoCapture(input_video_path)
    mask_cap = cv2.VideoCapture(mask_video_path)
    
    if not cap.isOpened() or not mask_cap.isOpened():
        print("Error: Could not open video files.")
        return
    
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (512, 512))
    
    while cap.isOpened() and mask_cap.isOpened():
        ret, frame = cap.read()
        ret_mask, mask = mask_cap.read()
        
        if not ret or not ret_mask:
            break
        
    
        # converted_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
        # _, thresh_binary = cv2.threshold(converted_mask, 2, 255, cv2.THRESH_BINARY)
        
        # cv2.imshow('mask', mask)
        # cv2.waitKey(1)
        
        cropped_image, bbox = crop_frame_square(frame, cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY))
        rescaled_image = animaimg.rescale_image(cropped_image, 512)
        # print(bbox)
        bbox_array.append(bbox)
        # stitched_image = restitch_frame(frame, cropped_image, bbox)
        
        out.write(rescaled_image)
    
    cap.release()
    mask_cap.release()
    out.release()
    cv2.destroyAllWindows()
    # WRITE BBOX_ARRAY TO FILE
    np.save(output_bbox_path, np.array(bbox_array))

In [193]:
def restitch_cropped_to_video(bg_video_path, fg_video_path, output_video_path, bbox_array_path):
    bg_cap = cv2.VideoCapture(bg_video_path)
    fg_cap = cv2.VideoCapture(fg_video_path)
    # frame_number = 0

    
    if not bg_cap.isOpened() or not fg_cap.isOpened():
        print("Error: Could not open video files.")
        return
    
    frame_width = int(bg_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(bg_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = bg_cap.get(cv2.CAP_PROP_FPS)
    
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))
    
    bbox_array = np.load(bbox_array_path)
    
    while bg_cap.isOpened() and fg_cap.isOpened():
        ret, bg = bg_cap.read()
        ret_fg, fg = fg_cap.read()
        # print(frame_number)
        
        if not ret or not ret_fg:
            break
        
        stitched_frame = restitch_frame(bg, fg, bbox_array[0])
        out.write(stitched_frame)
        # frame_number += 1
        bbox_array = bbox_array[1:]

In [191]:
crop_video_to_mask('test_images/car/blender/car_seq_2_24.mp4', 'test_images/car/blender/car_seq_mask_24.mp4', 'test_images/output/output.mp4', 'test_images/output/output.npy')
print(bbox_array)



[(469, 18, 897), (468, 17, 897), (468, 15, 896), (467, 14, 897), (466, 12, 897), (466, 11, 896), (466, 10, 896), (466, 8, 896), (466, 7, 896), (466, 5, 896), (466, 4, 896), (467, 4, 895), (468, 2, 894), (469, 2, 893), (470, 1, 892), (472, 1, 891), (473, 0, 890), (475, 0, 889), (476, 0, 888), (478, 0, 887), (479, 0, 887), (480, 0, 886), (482, 0, 884), (483, 0, 884), (484, 0, 883), (485, 0, 882), (486, 0, 881), (487, 0, 880), (488, 0, 879), (488, 0, 878), (488, 0, 878), (488, 0, 878), (488, 0, 877), (487, 0, 877), (486, 0, 877), (486, 0, 876), (484, 0, 877), (483, 0, 877), (482, 0, 877), (481, 0, 877), (480, 0, 878), (478, 0, 879), (478, 0, 878), (476, 0, 879), (475, 0, 880), (475, 0, 879), (473, 0, 880), (472, 0, 880), (471, 0, 880), (470, 0, 880), (468, 0, 881), (468, 0, 880), (467, 0, 880), (466, 0, 880), (465, 0, 880), (465, 0, 880), (465, 0, 879), (464, 0, 879), (464, 0, 879), (464, 0, 878), (464, 0, 878), (464, 0, 877), (465, 0, 876), (465, 0, 876), (466, 1, 875), (468, 1, 874), (4

In [194]:
restitch_cropped_to_video('test_images/car/blender/car_seq_2_24.mp4', 'test_images/output/output.mp4','test_images/output/stitched_output.mp4' , 'test_images/output/output.npy')